### Creación de funciones para Render - FastApi

#### Importamos librerías

Comienza instalando solo las librerías que necesitas para empezar tu proyecto.

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

### EndPoint 1

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [2]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP1 = pq.read_table("../0-DATA/1desarrollador.parquet").to_pandas()
df_EP1.head(5)

,price,developer,item_id,año
0,9.99,valve,70,2015
1,9.99,valve,70,2011
2,9.99,valve,70,2014
3,9.99,valve,70,2013
4,9.99,valve,70,2013


In [3]:
def developer(desarrollador: str, df_EP1: pd.DataFrame):
    # Convertimos el nombre del desarrollador a minúsculas para evitar errores de coincidencia de mayúsculas y minúsculas
    desarrollador = desarrollador.lower()
    
    # Obtenemos la lista única de desarrolladores en el DataFrame proporcionado
    devel = df_EP1.developer.unique()
    
    # Comprobamos si el desarrollador proporcionado está en la lista de desarrolladores del DataFrame
    if desarrollador not in devel:
        return {"message": "Mi base de datos no tiene registros de ese desarrollador"}

    # Obtenemos una lista de años únicos en el DataFrame
    listaaños = df_EP1.año.unique()
    
    # Obtenemos una lista de desarrolladores únicos en el DataFrame
    listadeveloper = df_EP1.developer.unique()
    
    # Creamos una lista vacía para almacenar los resultados buscados
    lista = []
    
    # Comprobamos si el desarrollador proporcionado está en la lista de desarrolladores únicos
    if desarrollador in listadeveloper:
        # Iteramos sobre los años únicos
        for años in listaaños:
            # Contamos la cantidad total de elementos del desarrollador en el año actual
            todo = len(df_EP1[(df_EP1.año == años) & (df_EP1.developer == desarrollador)])
            
            # Contamos la cantidad de elementos gratuitos del desarrollador en el año actual
            cantidaditems = len(df_EP1[(df_EP1.price == 0) & (df_EP1.año == años) & (df_EP1.developer == desarrollador)])
            
            # Verificamos si hay elementos en el año actual o si hay elementos gratuitos para evitar divisiones por cero
            if todo != 0 or cantidaditems != 0:
                # Calculamos el porcentaje de elementos gratuitos respecto al total de elementos
                porcentaje = f"{round(cantidaditems*100/todo,2)}%"
                
                # Añadimos los datos del año actual a la lista
                lista.append([años,cantidaditems,porcentaje])
            else:
                pass
    
    # Creamos un DataFrame a partir de la lista de resultados
    df = pd.DataFrame(lista, columns=["Año", "Cantidad de Items", "Contenido Free"])
    
    # Agregamos un encabezado a las columnas del DataFrame
    df.columns = ["Año", "Cantidad de Items", "Contenido Free"]
    
    # Ordenamos el DataFrame por año de forma descendente
    df = df.sort_values(by='Año',ascending=False)
    
    # Devolvemos el DataFrame resultante
    return df

In [4]:
developer('valve', df_EP1) # 'Valve', 'Trion Worlds', 'Cryptic Studios', 'Grey Havens',
                                   #'Three Rings', 'Daybreak Game Company', 'Reloaded Productions',
                                   #'Splash Damage', 'CCP', 'Nimbly Games', 'Team Chivalry',
                                   #'PopCap Games, Inc.'otras empresas

,Año,Cantidad de Items,Contenido Free
0,2015,676,24.9%
2,2014,1609,45.92%
3,2013,817,56.38%
4,2012,275,66.11%
1,2011,88,53.33%
5,2010,15,68.18%


### EndPoint 2

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [5]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP2 = pq.read_table("../0-DATA/2User_id.parquet").to_pandas()
df_EP2.head(5)

,price,item_id,recommend,user_id
0,9.99,70,1,eizanaratofujimaki
1,9.99,70,1,gamerfag
2,9.99,70,1,76561198020928326
3,9.99,70,1,bluegills
4,9.99,70,1,76561198071955492


In [6]:
def userdata(usuario: str, df_EP2: pd.DataFrame):
    # Convertimos el nombre del usuario a minúsculas para evitar errores de coincidencia de mayúsculas y minúsculas
    usuario = usuario.lower()
    
    # Obtenemos la lista única de IDs de usuario en el DataFrame proporcionado
    user = df_EP2.user_id.unique()
    
    # Comprobamos si el usuario proporcionado está en la lista de IDs de usuario del DataFrame
    if usuario not in user:
        return {"message": "Mi base de datos no tiene registros de ese usuario"}
    
    # Obtenemos una lista de IDs de usuario únicos en el DataFrame
    lista = df_EP2.user_id.unique()
    
    # Filtramos el DataFrame para obtener los datos del usuario proporcionado
    dfusuario = df_EP2[df_EP2.user_id == usuario]
    
    # Calculamos la cantidad total de dinero gastado por el usuario
    maneyuser = f"{round(dfusuario.price.sum(), 2)} USD"
    
    # Calculamos el porcentaje de recomendación del usuario (porcentaje de recomendaciones positivas)
    recomend = f"{int(len(dfusuario[dfusuario.recommend == 1]) * 100 / len(dfusuario))}%"
    
    # Contamos la cantidad total de elementos que tiene el usuario
    itemss = len(dfusuario)
    
    # Creamos un diccionario con la información del usuario
    data = {"User": usuario, 
            "Dinero gastado": maneyuser, 
            "% de recomendación": recomend, 
            "cantidad de items": itemss}
    
    # Devolvemos el diccionario con la información del usuario
    return data 


In [9]:
userdata('eizanAratoFujimaki', df_EP2)#'EizanAratoFujimaki', 'GamerFag', '76561198020928326', 'Bluegills',
                            #'76561198071955492', 'digfernandes', '76561197970812298',
                            #'meeeedie', 'phrostb', 'KewlKatzz', 'TheRealMeatyMate',
                            #'Auditore70', 'voaDs', '76561198060319088', '76561198085571748' u otros usuarios

{'User': 'eizanaratofujimaki',
 'Dinero gastado': '9.99 USD',
 '% de recomendación': '100%',
 'cantidad de items': 1}

### EndPoint 3

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [10]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP3 = pq.read_table("../0-DATA/3genero.parquet").to_pandas()
df_EP3.head(5)

,genres_clean,user_id,año,playtime_hours
0,action,EizanAratoFujimaki,2015,23.25
1,action,GamerFag,2011,9.83
2,action,76561198020928326,2014,93.32
3,action,Bluegills,2013,1.07
4,action,76561198071955492,2013,53.92


In [11]:
def UserForGenre(genero: str):
    # Convertimos la primera letra del género en mayúscula y el resto en minúscula
    generoO = genero.capitalize()
    genero = genero.lower()
    
    # Obtenemos la lista de géneros únicos en el DataFrame
    gender = df_EP3.genres_clean.unique()
    
    # Comprobamos si el género proporcionado está en la lista de géneros del DataFrame
    if genero not in gender:
        return {"message": "Mi base de datos no tiene registros de ese genero"}

    # Obtenemos la lista de años únicos en el DataFrame
    años = df_EP3.año.unique()
    
    # Inicializamos las variables para almacenar al usuario con más horas jugadas y las horas totales
    usuario_total = None
    horas_totales_usuario = 0
    user_data = []

    # Iteramos sobre los años únicos
    for añi in años:
        # Filtramos los datos para el género específico y el año actual
        df_genero = df_EP3[(df_EP3.genres_clean == genero) & (df_EP3.año == añi)]

        # Encontramos el usuario con más horas de juego
        if not df_genero.empty:
            usuario = df_genero.groupby('user_id')["playtime_hours"].sum().idxmax()
            horasT = df_genero[df_genero.user_id == usuario]["playtime_hours"].sum()

            # Actualizamos el usuario total si es necesario
            if horasT > horas_totales_usuario:
                usuario_total = usuario
                horas_totales_usuario = horasT

            # Añadimos los datos para este año a la lista
            user_data.append({"Año": añi, "Horas": horasT})
        else:
            # No se encontraron datos para este año, añadimos un diccionario con horas igual a cero
            user_data.append({"Año": añi, "Horas": 0})

    # Creamos un diccionario de respuesta con el usuario con más horas jugadas y las horas jugadas por año
    respuesta = {
        "Usuario con más horas jugadas para Género --> " + generoO: usuario_total,
        "Horas jugadas": user_data,
    }

    # Devolvemos la respuesta
    return respuesta


In [14]:
UserForGenre('action')#'Action', 'Indie', 'Racing', 'Strategy', 'Casual', 'Simulation',
        #'Adventure', 'RPG', 'Sports', 'Free to Play',
        #'Animation &amp; Modeling', 'Utilities', 'Massively Multiplayer',
        #'Design &amp; Illustration', 'Video Production'

{'Usuario con más horas jugadas para Género --> Action': 'shinomegami',
 'Horas jugadas': [{'Año': 2015, 'Horas': 8848.03},
  {'Año': 2011, 'Horas': 2247.96},
  {'Año': 2014, 'Horas': 7107.580000000001},
  {'Año': 2013, 'Horas': 6042.35},
  {'Año': 2012, 'Horas': 3545.17},
  {'Año': 2010, 'Horas': 834.57}]}

### Pregunnta 4

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [15]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP4 = pq.read_table("../0-DATA/4año.parquet").to_pandas()
df_EP4.head(5)

,developer,recommend,user_id,sentiment_analysis,año
0,Valve,1,EizanAratoFujimaki,1,2015
1,Valve,1,GamerFag,0,2011
2,Valve,1,76561198020928326,1,2014
3,Valve,1,Bluegills,2,2013
4,Valve,1,76561198071955492,2,2013


In [16]:
def best_developer_year(year: int, df_EP4: pd.DataFrame):
    # Definimos el rango de años aceptado en la base de datos
    rango_aceptado = range(2010, 2016)
    
    # Comprobamos si el año proporcionado está dentro del rango aceptado
    if year not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}
    
    # Filtramos los datos para obtener las recomendaciones positivas y el análisis de sentimientos igual a 2 para el año proporcionado
    recom = df_EP4[(df_EP4.recommend == 1) & (df_EP4.sentiment_analysis == 2) & (df_EP4.año == year)]
    
    # Agrupamos por desarrollador y contamos las recomendaciones para cada uno
    developer_recommendations = recom.groupby("developer")["recommend"].sum()
    
    # Obtenemos los 3 mejores desarrolladores con más recomendaciones
    top_3_developers = developer_recommendations.nlargest(3)

    # Creamos una lista para almacenar los 3 mejores desarrolladores
    top_developers_list = []
    
    # Iteramos sobre los 3 mejores desarrolladores y su cantidad de recomendaciones
    for rank, (developer, count) in enumerate(top_3_developers.items(), start=1):
        # Añadimos el desarrollador y su puesto a la lista
        top_developers_list.append({"Puesto " + str(rank): developer})

    # Devolvemos la lista de los 3 mejores desarrolladores
    return top_developers_list



In [19]:
best_developer_year(2015, df_EP4) # 2010,2011,2012,2013,2014,2015

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Facepunch Studios'},
 {'Puesto 3': 'Smartly Dressed Games'}]

### Pregunta 5

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [20]:
#Leemos el archivo reviews.parquet en un DataFrame
df_EP5 = pq.read_table("../0-DATA/5desarrolladora.parquet").to_pandas()
df_EP5.head(5)

,developer,sentiment_analysis
0,valve,1
1,valve,0
2,valve,1
3,valve,2
4,valve,2


In [21]:
def developer_reviews_analysis(desarrolladora: str, df_EP5: pd.DataFrame):
    # Convertimos el nombre de la desarrolladora a minúsculas para evitar errores de coincidencia de mayúsculas y minúsculas
    desarrolladora = desarrolladora.lower()
    
    # Obtenemos la lista única de desarrolladoras en el DataFrame proporcionado
    develo = df_EP1.developer.unique()
    
    # Comprobamos si la desarrolladora proporcionada está en la lista de desarrolladoras del DataFrame
    if desarrolladora not in develo:
        return {"message": "Mi base de datos no tiene registros de ese desarrollador"}
    
    # Filtramos el DataFrame para obtener solo las reseñas de la desarrolladora proporcionada
    df_filt_developer = df_EP5[(df_EP5['developer'] == desarrolladora) & (df_EP5['sentiment_analysis'] != 1)]
    
    # Verificamos si hay datos para la desarrolladora proporcionada
    if not df_filt_developer.empty:
        # Contamos los diferentes sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filt_developer['sentiment_analysis'].value_counts()
        sentiment_mapping = {0: 'Negative', 2: 'Positive'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        # Creamos un diccionario con el nombre de la desarrolladora y los recuentos de sentimientos
        result = {desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    # Devolvemos el resultado
    return result


In [24]:
developer_reviews_analysis('fUncom',df_EP5) #'Valve', 'Outerlight Ltd.', 'GlyphX Games',
                            #'Introversion Software', 'Facepunch Studios',
                            #'Bugbear Entertainment', 'Funcom',
                            #'Firaxis Games,Feral Interactive (Mac)',
                            #'Crystal Dynamics,Feral Interactive (Mac)', 'CAPCOM Co., Ltd.',
                            #'id Software', 'Gray Matter Studios', '2K Boston,2K Australia',
                            #'Relic Entertainment', 'The Creative Assembly'

{'funcom': {'Positive': 5}}